---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [2]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing
import collections


def blight_model():
    
    train_all = pd.read_csv('train.csv',encoding = "ISO-8859-1")
    test_all = pd.read_csv('test.csv',encoding = "ISO-8859-1")
    
    test_id = np.array(test_all['ticket_id'])
    ######################## Preprocessing #######################################
    train1 = train_all[np.isfinite(train_all['compliance'])]
    #train1 = train_all.dropna(subset=['compliance'])

    #drop features that exist only in training set
    train2 = train1.drop(['payment_amount', 'balance_due','payment_date', 'payment_status', 'collection_status','compliance_detail'], axis=1)
    
    ##check if the all the entries of a feature are the same 
    ##e.g. admin_fee, state_fee, country(only USA in test)
    #arr = train2.loc[:,'country'].as_matrix().astype(str)
    #arr_test = test_all.loc[:,'country'].as_matrix().astype(str)
    ##print(arr.isnull().sum(),arr_test.isnull().sum())
    #print(np.intersect1d(arr,arr_test))
    ###check_arr = arr[0]*np.ones(len(train2))
    ###print(np.array_equal(arr,check_arr))
    #print(np.where(arr != arr[0])) # for both int and string data, check if it returns an empty array
    #print(collections.Counter(arr))
    #print(collections.Counter(arr_test))
        
    #drop extra uninformative features from both training and test set
    train = train2.drop(['inspector_name', 'violation_zip_code', 'violation_street_number','violation_street_name','violator_name',\
                    'mailing_address_str_number', 'mailing_address_str_name', 'city',\
                    'state', 'zip_code', 'non_us_str_code', 'country','ticket_issued_date', 'hearing_date',\
                    'admin_fee', 'state_fee', 'clean_up_cost', 'grafitti_status','violation_description'], axis=1)
    test = test_all.drop(['inspector_name', 'violation_zip_code', 'violation_street_number','violation_street_name','violator_name',\
                    'mailing_address_str_number', 'mailing_address_str_name', 'city',\
                    'state', 'zip_code', 'non_us_str_code', 'country','ticket_issued_date', 'hearing_date',\
                    'admin_fee', 'state_fee', 'clean_up_cost', 'grafitti_status','violation_description'], axis=1)
    ################################################################################################
    train = train.set_index('ticket_id')
    test = test.set_index('ticket_id')
    
    #y_train = train.iloc[:,-1]
    #x_train = train.iloc[:,:-1]
    # Convert string columns to categorical
    cols = test.select_dtypes(exclude=['float', 'int']).columns
    #print(cols)
    len_train = len(train)
    #print(train.loc[:,'violation_code'])
    temp_concat = pd.concat((train[cols], test[cols]), axis=0)
    # Some filtering on violation_code to make it more manageable
    temp_concat['violation_code'] = temp_concat['violation_code'].apply(lambda x: x.split(' ')[0])
    temp_concat['violation_code'] = temp_concat['violation_code'].apply(lambda x: x.split('(')[0])
    temp_concat['violation_code'][temp_concat['violation_code'].apply(lambda x: x.find('-')<=0)] = np.nan
    #print(temp_concat.loc[:,'violation_code'])

    #print(temp_concat)
    # Make all codes with < 10 occurrences null
    #counts = temp_concat['violation_code'].value_counts()
    #temp_concat['violation_code'][temp_concat['violation_code'].isin(counts[counts < 10].index)] = np.nan
    for column_name in cols:
        #print'Converting to categorical...', column_name, '# variables:', len(temp_concat[column_name].unique())
        dummies = pd.get_dummies(temp_concat[column_name])
        temp_concat[dummies.columns] = dummies
        temp_concat.drop(column_name, axis=1, inplace=True)
        train.drop(column_name, axis=1, inplace=True)
        test.drop(column_name, axis=1, inplace=True)

    train[temp_concat.columns] = temp_concat.loc[train.index]
    test[temp_concat.columns] = temp_concat.loc[test.index]
    features = list( test.columns )
    response = ['compliance']
    #print(features)
    x_train = train[features]
    y_train = np.array(train[response]).ravel()

    #y_train = train.iloc[:,-1]
    #x_train = train.iloc[:,:-1]
    #print(x_train.axes)
    #print(y_train.axes)
    x_train = x_train.replace([np.inf, -np.inf], np.nan)
    #x_train[pd.isnull(x_train)] = 0

    #print(y_train)
    x_test = test[features]
    #print(x_train)
    #sg = SGDClassifier().fit(x_train,y_train)
    #acc_train = sg.score(x_train, y_train)

    nb = GaussianNB().fit(x_train, y_train)
    acc_train = nb.score(x_train, y_train)
    print(acc_train)
    res = nb.predict_proba(x_test)
    ser = pd.Series(res[:,1],index=test_id)
    return ser#test_id, res[:,1]
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.807355516637


284932     1.000000e+00
285362     1.148512e-07
285361     9.999763e-01
285338     2.030096e-04
285346     7.763583e-01
285345     2.030096e-04
285347     9.464534e-01
285342     9.994127e-01
285530     1.148512e-07
284989     5.669441e-05
285344     4.233899e-04
285343     1.148512e-07
285340     1.148512e-07
285341     4.233899e-04
285349     7.763583e-01
285348     2.030096e-04
284991     5.669441e-05
285532     5.669441e-05
285406     5.669441e-05
285001     2.946127e-05
285006     1.270446e-68
285405     1.148512e-07
285337     5.669441e-05
285496     4.233899e-04
285497     2.030096e-04
285378     1.148512e-07
285589     5.669441e-05
285585     2.030096e-04
285501     9.999763e-01
285581     1.148512e-07
              ...      
376367     5.336272e-67
376366     9.688030e-05
376362     9.987179e-01
376363     1.000000e+00
376365     5.336272e-67
376364     9.688030e-05
376228     9.688030e-05
376265     9.688030e-05
376286     8.029112e-01
376320     9.688030e-05
376314     9.688